In [43]:
import pandas as pd
pd.options.display.max_rows = 100

df = pd.concat(
    [
        pd.read_csv(
            f"data/OraclesElixir/{year}_LoL_esports_match_data_from_OraclesElixir.csv",
            dtype={"url": "str"}
        )
        for year in range(2020, 2025)
    ],
    ignore_index=True
)

rows, cols = df.shape
print(f"The CSV file has {rows} rows and {cols} columns.")

print(df.columns.tolist())


The CSV file has 666168 rows and 161 columns.
['gameid', 'datacompleteness', 'url', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'participantid', 'side', 'position', 'playername', 'playerid', 'teamname', 'teamid', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'infernals', 'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'void_grubs', 'opp_void_grubs', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'turretplates', 'opp_turretplates', 'inhibitor

In [44]:
num_complete_rows = df[df["datacompleteness"] == 'complete'].shape[0]
total_rows = df.shape[0]
ratio = num_complete_rows / total_rows
print(f"Number of rows where datacompleteness is 'complete': {num_complete_rows}")
print(f"Ratio of 'complete' rows to total rows: {ratio:.4f}")

df = df[df["datacompleteness"] == 'complete']


Number of rows where datacompleteness is 'complete': 590184
Ratio of 'complete' rows to total rows: 0.8859


In [45]:
for i, col in enumerate(df.columns):
    print(f"Column {i}: {col}")
    print(df[col].describe())
    if df[col].dtype == 'object' and df[col].nunique() <= 20:
        print("Unique values:", df[col].unique())
    print("------------------------------------")

Column 0: gameid
count               590184
unique               49182
top       LOLTMNT02_194401
freq                    12
Name: gameid, dtype: object
------------------------------------
Column 1: datacompleteness
count       590184
unique           1
top       complete
freq        590184
Name: datacompleteness, dtype: object
Unique values: ['complete']
------------------------------------
Column 2: url
count                                          248580
unique                                          19100
top       https://lpl.qq.com/es/stats.shtml?bmid=6904
freq                                               60
Name: url, dtype: object
------------------------------------
Column 3: league
count     590184
unique        89
top          LCK
freq       28848
Name: league, dtype: object
------------------------------------
Column 4: year
count    590184.000000
mean       2021.937335
std           1.394531
min        2020.000000
25%        2021.000000
50%        2022.000000
75%      

In [46]:
team_rows = df[df['position'] == 'team'].copy()
player_rows = df[df['position'] != 'team']

positions = ['top', 'jng', 'mid', 'bot', 'sup']

for pos in positions:
    champ_col = (
        player_rows[player_rows['position'] == pos]
        .loc[:, ['gameid', 'side', 'champion']]
        .rename(columns={'champion': f'{pos}_champ'})
    )
    
    team_rows = team_rows.merge(champ_col, on=['gameid', 'side'], how='left')
df = team_rows


Dropping Data

In [47]:
columns_to_drop = (
    df.columns[1:11]  # Metadata columns
    .union(df.columns[12:18])  # Additional metadata columns
    .union(df.columns[18:28])  # BP data
    .union(df.columns[30:43])  # End game data columns
    .union(df.columns[48:57])  # Drake-related columns
    .union(df.columns[40:43])  # Individual data columns
    .union(pd.Index([df.columns[78]]))  # Specific column (xpat15)
    .union(pd.Index([df.columns[91]]))  # Specific column (opp_deathsat15)
    .union(pd.Index([df.columns[95]]))  # Specific column (bot_champ)
    .union(pd.Index([df.columns[28]]))  # Specific column (firstbaron)
    .union(df.columns[131:161])  # Data after 20 minutes
)

df.drop(columns=columns_to_drop, axis=1, inplace=True)



In [48]:
df.head()

,gameid,side,result,team kpm,ckpm,firstdragon,dragons,opp_dragons,elders,opp_elders,...,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,top_champ,jng_champ,mid_champ,bot_champ,sup_champ
0,ESPORTSTMNT03/1241318,Blue,1,0.4595,0.7568,0.0,2.0,3.0,0.0,0.0,...,0.0,1.0,1.0,2.0,0.0,Rumble,Elise,Qiyana,Miss Fortune,Nautilus
1,ESPORTSTMNT03/1241318,Red,0,0.2973,0.7568,1.0,3.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,Aatrox,Gragas,Mordekaiser,Xayah,Rakan
2,ESPORTSTMNT03/1241322,Blue,0,0.1347,0.7005,0.0,1.0,4.0,0.0,0.0,...,0.0,6.0,6.0,7.0,0.0,Jayce,Jarvan IV,Orianna,Miss Fortune,Nautilus
3,ESPORTSTMNT03/1241322,Red,1,0.5658,0.7005,1.0,4.0,1.0,0.0,0.0,...,7.0,0.0,0.0,0.0,6.0,Aatrox,Elise,Qiyana,Varus,Tahm Kench
4,ESPORTSTMNT03/1241324,Blue,1,0.7364,0.8767,1.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Renekton,Rek'Sai,Mordekaiser,Xayah,Rakan


In [49]:
# Print the dataframe information
print(df.info())

# Print the dataframe description
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98364 entries, 0 to 98363
Data columns (total 84 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gameid                    98364 non-null  object 
 1   side                      98364 non-null  object 
 2   result                    98364 non-null  int64  
 3   team kpm                  98364 non-null  float64
 4   ckpm                      98364 non-null  float64
 5   firstdragon               98356 non-null  float64
 6   dragons                   98364 non-null  float64
 7   opp_dragons               98364 non-null  float64
 8   elders                    98364 non-null  float64
 9   opp_elders                98364 non-null  float64
 10  firstherald               98364 non-null  float64
 11  heralds                   95204 non-null  float64
 12  opp_heralds               95204 non-null  float64
 13  void_grubs                16858 non-null  float64
 14  opp_vo

In [50]:
# Concatenate the head and tail of the dataframe
head_and_tail = pd.concat([df.head(), df.tail()])

# Save to a CSV file
head_and_tail.to_csv("head_and_tail.csv", index=False)

In [51]:
# Define null_columns if not already defined
null_counts = df.isnull().sum()
null_columns = null_counts[null_counts > 0]

# Initialize an empty DataFrame to store the rows
rows_with_nulls = pd.DataFrame()

# Iterate through columns with null values
for col in null_columns.index:
    # Select rows where the column has null values
    null_rows = df[df[col].isnull()]
    
    # Append the first two rows with null values for the column
    rows_with_nulls = pd.concat([rows_with_nulls, null_rows.head(2)], ignore_index=True)

# Display the resulting DataFrame
rows_with_nulls.head()
rows_with_nulls.to_csv("rows_with_nulls.csv", index=False)


In [55]:
# Print the count of null values in each column
print("Null values in each column:")
null_counts = df.isnull().sum()
null_columns = null_counts[null_counts > 0]
print(null_columns)

# Calculate the ratio of null values for each column
null_ratio = (null_counts / total_rows)

# Filter and print only the columns where the ratio of null values is greater than 0
null_columns_with_ratio = null_ratio[null_ratio > 0]
print("Columns with null values and their ratios:")
print(null_columns_with_ratio)




Null values in each column:
firstdragon                    8
heralds                     3160
opp_heralds                 3160
void_grubs                 81506
opp_void_grubs             81506
firstbaron                   436
firsttower                    22
firstmidtower                 18
turretplates               39006
opp_turretplates           39006
inhibitors                   116
opp_inhibitors               116
damagetochampions             22
dpm                           22
damagetakenperminute          22
wardsplaced                   22
wpm                           22
wardskilled                   22
wcpm                          22
controlwardsbought            22
visionscore                 3044
vspm                        3044
earnedgold                    22
earned gpm                    22
minionkills                 2650
monsterkills                  22
monsterkillsownjungle      59380
monsterkillsenemyjungle    59380
cspm                        2628
goldat10       

In [53]:
# Print the count of unique values in each column
print("Unique values in each column:")
print(df.nunique())

Unique values in each column:
gameid                      49182
side                            2
result                          2
team kpm                    11143
ckpm                        11909
firstdragon                     2
dragons                         8
opp_dragons                     8
elders                          4
opp_elders                      4
firstherald                     2
heralds                         3
opp_heralds                     3
void_grubs                      7
opp_void_grubs                  7
firstbaron                      2
barons                          6
opp_barons                      6
firsttower                      2
towers                         12
opp_towers                     12
firstmidtower                   2
firsttothreetowers              2
turretplates                   16
opp_turretplates               16
inhibitors                     11
opp_inhibitors                 11
damagetochampions           58620
dpm               

In [57]:
correlation = df['void_grubs'].corr(df['result'])  # win should be 0/1
print("Correlation with win:", correlation)

Correlation with win: 0.1548435396610444
